<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/main/06_QA_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information

Reference: 
* [End-To-End Memory Networks](https://arxiv.org/pdf/1503.08895.pdf)

# import data

In [4]:
import numpy as np
import pickle

In [18]:
with open("/content/drive/MyDrive/01_Personal/01_Study/01_ML/04_QAChatBot/train_qa.txt", "rb") as f:
  train_data = pickle.load(f)